In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from datetime import datetime
import lightgbm as lgb

In [18]:
# Load train and test data
dataset_fe = pd.read_csv('../../datasets_procesados/dataset_procesado.csv')

In [4]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 35)
dataset_fe

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Description_limpia,Removed_Special_Chars,longitud_descripcion,cantidad_palabras_no_stopwords,stopwords_eliminadas,descripcion_para_analisis,palabras_caracteres_eliminados,Nombres_limpios,rescatados_por_este_rescatista,disponibilidad_imagen
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,nibble is month old ball of cuteness he is ene...,"['T', 'L', 'I', 'H', '+', '.', '3', ""'"", 'N']",63,30,"['is', 'of', 'he', 'is', 'and', 'of', 'few', '...","['nibble', 'month', 'old', 'ball', 'cuteness',...","['T', 'L', 'I', 'H', '+', '.', '3', ""'"", 'N', ...",nibble,9,3
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,just found it alone yesterday near my apartmen...,"['.', 'I']",21,10,"['found', 'it', 'alone', 'my', 'it', 'was', 's...","['just', 'yesterday', 'near', 'apartment', 'sh...","['.', 'I', 'found', 'it', 'alone', 'my', 'it',...",no name yet,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,2,Ms Daym,9,266,0,2,4,7,0,1,1,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,she is very shy adventures and independent she...,['.'],22,11,"['she', 'is', 'very', 'and', 'she', 'but', 'an...","['shy', 'adventures', 'independent', 'just', '...","['.', 'she', 'is', 'very', 'and', 'she', 'but'...",ms daym,5,3
14992,1,Fili,1,307,307,1,2,0,0,2,1,2,2,2,1,1,0,41332,9ed1d5493d223eaa5024c1a031dbc9c2,0,Fili just loves laying around and also loves b...,a83d95ead,1.0,3,fili just loves laying around and also loves b...,"['V', '.', 'F', ';']",16,8,"['around', 'and', 'also', 'being', 'under', 't...","['fili', 'just', 'loves', 'laying', 'loves', '...","['V', '.', 'F', ';', 'around', 'and', 'also', ...",fili,7,3


In [19]:
# Agrupar por RescuerID y sumar Quantity por adoption speed
grouped = dataset_fe.groupby(["RescuerID", "AdoptionSpeed"])["Quantity"].sum().unstack(fill_value=0)

# Convert column names to strings (important if some columns are integers)
grouped.columns = grouped.columns.astype(str)

grouped["Quantity_Total"] = grouped.sum(axis=1)

for speed_col in grouped.columns.drop("Quantity_Total"):
    grouped[f"ratio_AdoptionSpeed{speed_col}"] = grouped[speed_col] / grouped["Quantity_Total"]

# Filter only the new ratio columns by checking for the substring
new_ratio_cols = [col for col in grouped.columns if "ratio_AdoptionSpeed" in col]

dataset_fe = dataset_fe.merge(grouped[new_ratio_cols],
                              left_on="RescuerID",
                              right_index=True,
                              how="left")

In [20]:
dataset_fe

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,...,PhotoAmt,AdoptionSpeed,Description_limpia,Removed_Special_Chars,longitud_descripcion,cantidad_palabras_no_stopwords,stopwords_eliminadas,descripcion_para_analisis,palabras_caracteres_eliminados,Nombres_limpios,rescatados_por_este_rescatista,disponibilidad_imagen,ratio_AdoptionSpeed0,ratio_AdoptionSpeed1,ratio_AdoptionSpeed2,ratio_AdoptionSpeed3,ratio_AdoptionSpeed4
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,...,1.0,2,nibble is month old ball of cuteness he is ene...,"['T', 'L', 'I', 'H', '+', '.', '3', ""'"", 'N']",63,30,"['is', 'of', 'he', 'is', 'and', 'of', 'few', '...","['nibble', 'month', 'old', 'ball', 'cuteness',...","['T', 'L', 'I', 'H', '+', '.', '3', ""'"", 'N', ...",nibble,9,3,0.0,0.111111,0.666667,0.222222,0.000000
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,...,2.0,0,just found it alone yesterday near my apartmen...,"['.', 'I']",21,10,"['found', 'it', 'alone', 'my', 'it', 'was', 's...","['just', 'yesterday', 'near', 'apartment', 'sh...","['.', 'I', 'found', 'it', 'alone', 'my', 'it',...",no name yet,1,3,1.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,2,Ms Daym,9,266,0,2,4,7,0,1,1,1,1,1,1,1,0,...,3.0,4,she is very shy adventures and independent she...,['.'],22,11,"['she', 'is', 'very', 'and', 'she', 'but', 'an...","['shy', 'adventures', 'independent', 'just', '...","['.', 'she', 'is', 'very', 'and', 'she', 'but'...",ms daym,5,3,0.0,0.000000,0.400000,0.200000,0.400000
14992,1,Fili,1,307,307,1,2,0,0,2,1,2,2,2,1,1,0,...,1.0,3,fili just loves laying around and also loves b...,"['V', '.', 'F', ';']",16,8,"['around', 'and', 'also', 'being', 'under', 't...","['fili', 'just', 'loves', 'laying', 'loves', '...","['V', '.', 'F', ';', 'around', 'and', 'also', ...",fili,7,3,0.0,0.000000,0.000000,0.714286,0.285714


In [8]:
# Define the output path
output_path = '../../datasets_procesados/dataset_procesado_con_ratios.csv'

# Save the DataFrame to CSV
dataset_fe.to_csv(output_path, index=False)

print(f"Dataset saved successfully to {output_path}")

Dataset saved successfully to ../../datasets_procesados/dataset_procesado_con_ratios.csv
